# 2048 Game Using Deep Q-Network

By Abdallah Saba & Mahmoud Ghandour

Installing necessary dependencies

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install stable_baselines3
!pip install shimmy
!pip install manim
!pip install IPython --upgrade

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
l

# Creating the Agent, Environment, and the Rewards for the game

In [ ]:
from manim import *
import numpy as np
import gym
from stable_baselines3 import DQN
import gym.spaces as spaces
from gym.utils import seeding
from manim import config


config.disable_caching_warning = True


class Base2048Env(gym.Env):
    metadata = {'render.modes': ['human'],}
    LEFT = 0
    UP = 1
    RIGHT = 2
    DOWN = 3


    ACTION_STRING = {
        LEFT: 'left',
        UP: 'up',
        RIGHT: 'right',
        DOWN: 'down',
    }
    def __init__(self, width=4, height=4):
        self.width = width
        self.height = height


        self.observation_space = spaces.Box(low=2,
                                            high=2**32,
                                            shape=(self.width, self.height),
                                            dtype=np.int64)
        self.action_space = spaces.Discrete(4)


        # Internal Variables
        self.board = None
        self.np_random = None
        self.seed()
        self.reset()


    def seed(self, seed = None):
        self.np_random, seed = seeding.np_random(seed)
        return[seed]


    def all_moves(self, updated_obs1, reward_1, level):
        gamma=0.5
        rotated_obs11 = np.rot90(updated_obs1, k=0)
        reward11, updated_obs11 = self._slide_left_and_merge(rotated_obs11)
        rotated_obs12 = np.rot90(updated_obs1, k=1)
        reward12, updated_obs12 = self._slide_left_and_merge(rotated_obs12)
        rotated_obs13 = np.rot90(updated_obs1, k=2)
        reward13, updated_obs13 = self._slide_left_and_merge(rotated_obs13)
        rotated_obs14 = np.rot90(updated_obs1, k=3)
        reward14, updated_obs14 = self._slide_left_and_merge(rotated_obs14)
        reward11=(reward11*pow(gamma,level))+reward_1
        reward12=(reward12*pow(gamma,level))+reward_1
        reward13=(reward13*pow(gamma,level))+reward_1
        reward14=(reward14*pow(gamma,level))+reward_1


        rewardleft=[0.0] * 4
        if (level < 3):
            rewardleft[0] = self.all_moves(updated_obs11, reward11, level+1)
            rewardleft[1] = self.all_moves(updated_obs12, reward12, level+1)
            rewardleft[2] = self.all_moves(updated_obs13, reward13, level+1)
            rewardleft[3] = self.all_moves(updated_obs14, reward14, level+1)
        MaxReward1=max(rewardleft)
        MaxR=max(MaxReward1, reward14, reward13, reward12, reward11)
        return MaxR
    def step(self, k):
        #going to the left
        gamma=0.5
        rotated_obs1 = np.rot90(self.board, k=0)
        reward1, updated_obs1 = self._slide_left_and_merge(rotated_obs1)
        MaxReward1 = self.all_moves(updated_obs1, reward1, 1)
        #going to up
        rotated_obs2 = np.rot90(self.board, k=1)
        reward2, updated_obs2 = self._slide_left_and_merge(rotated_obs2)
        MaxReward2 = self.all_moves(updated_obs2, reward2, 1)


        # going to the right
        rotated_obs3 = np.rot90(self.board, k=2)
        reward3, updated_obs3 = self._slide_left_and_merge(rotated_obs3)
        MaxReward3 = self.all_moves(updated_obs3, reward3, 1)


        #going down
        rotated_obs4 = np.rot90(self.board, k=3)
        reward4, updated_obs4 = self._slide_left_and_merge(rotated_obs4)
        MaxReward4 = self.all_moves(updated_obs4, reward4, 1)


        if(MaxReward1>=MaxReward2 and MaxReward1>=MaxReward3 and MaxReward1 >=MaxReward4):
            maxaction=0
            updated_obs=updated_obs1
            maxreward=reward1
        elif (MaxReward2>=MaxReward3 and MaxReward2>=MaxReward4):
            maxaction=1
            updated_obs=updated_obs2
            maxreward=reward2
        elif (MaxReward3>=MaxReward4):
            maxaction=2
            updated_obs=updated_obs3
            maxreward=reward3
        else:
            maxaction=3
            updated_obs=updated_obs4
            maxreward=reward4
        self.board = np.rot90(updated_obs, k= 4-maxaction)
        # Place one random tile on empty location
        self._place_random_tiles(self.board, count=1)
        done = self.is_done()
        return self.board, maxreward, done, {}


    def is_done(self):
        copy_board = self.board.copy()
        if not copy_board.all():
            return False
        for maxaction in [0, 1, 2, 3]:
            rotated_obs = np.rot90(copy_board, k=maxaction)
            _, updated_obs = self._slide_left_and_merge(rotated_obs)
            if not updated_obs.all():
                return False
        return True
    def reset(self):
        """Place 2 tiles on empty board."""
        self.board = np.zeros((self.width, self.height), dtype=np.int64)
        self._place_random_tiles(self.board, count=2)
        return self.board


    def render(self, mode='human'):
        if mode == 'human':
            for row in self.board.tolist():
                print(' \t'.join(map(str, row)))
    def _sample_tiles(self, count=1):
        #Sample tile 2 or 4.
        choices = [2, 4]
        probs = [0.9, 0.1]
        tiles = self.np_random.choice(choices, size=count, p=probs)
        return tiles.tolist()
    def _sample_tile_locations(self, board, count=1):
        #Sample grid locations with no tile.
        zero_locs = np.argwhere(board == 0)
        zero_indices = self.np_random.choice(len(zero_locs), size=count)
        zero_pos = zero_locs[zero_indices]
        #zero_pos = list(zip(*zero_pos))
        return zero_pos


    def _place_random_tiles(self, board, count=1):
        if not board.all():
            tiles = self._sample_tiles(count)
            tile_locs = self._sample_tile_locations(board, count)
            #assert len(tiles) == len(tile_locs)
            #print(f"tiles location shape: {tile_locs}")
            #print(f"tiles shape: {tiles}")
            #print(f"tile board: {board}")
            for i in range(len(tile_locs)):
                if (len(tiles) == len(tile_locs)):
                    idx = tile_locs[i]
                    board[idx[0]][idx[1]] = tiles[i]
                else:
                    idx = {tile_locs[i][0] , tile_locs[i+1][0]}
                    board[idx[0]][idx[1]] = tiles[i]
            i=0
        # we need to create a recursive function that includes DQN
        # baby steps we create the normal recursive function without gamma
        # then we add gamma^n to the function
        #actions are columns and the rows are the states or location on puzzle
        #initialize a q table choose an action
        #perform action
        #measure rewards
        #update Q table
        #mean square error and regression model
    def _slide_left_and_merge(self, board):
        #Slide tiles on a grid to the left and merge.
        result = []
        score = 0
        for row in board:
            row = np.extract(row > 0, row)
            score_, result_row = self._try_merge(row)
            score += score_
            row = np.pad(np.array(result_row), (0, self.width - len(result_row)),
                         'constant', constant_values=(0,))
            result.append(row)
        return score, np.array(result, dtype=np.int64)
    @staticmethod
    def _try_merge(row):
        score = 0
        result_row = []
        i = 1
        while i < len(row):
            if row[i] == row[i - 1]:
                score += row[i] + row[i - 1]
                result_row.append(row[i] + row[i - 1])
                i += 2
            else:
                result_row.append(row[i - 1])
                i += 1
        if i == len(row):
            result_row.append(row[i - 1])
        return score, result_row

Manim Community v0.18.0

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


# Using MANIM (Mathematical Animation) library to visualize a single game that is played by our agent.

In [ ]:
class MatrixVisualization(Scene):
    def construct(self):
        # Initialize environment and RL model
        env = Base2048Env()
        env.seed(2048)
        rewards_sum=0
        moves_sum=0
        rewards_average=0
        moves_average=0
        rewards2_list = []
        """
        (policy, env, learning_rate=0.0001, buffer_size=1000000, learning_starts=50000, batch_size=32, tau=1.0, gamma=0.99,
        train_freq=4, gradient_steps=1, replay_buffer_class=None, replay_buffer_kwargs=None, optimize_memory_usage=False,
        target_update_interval=10000, exploration_fraction=0.1, exploration_initial_eps=1.0, exploration_final_eps=0.05,
        max_grad_norm=10, stats_window_size=100, tensorboard_log=None, policy_kwargs=None, verbose=0, seed=None, device='auto', _init_setup_model=True)
        """
        model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.0001, gamma=0.8, train_freq=4)
        model.learn(total_timesteps=1000, log_interval=4)


        # Number of iterations
        num_iterations = 1  # Number of games played
        #max_steps = 1500  # Maximum steps to animate

        for iteration in range(num_iterations):
            obs = env.reset()
            done = False
            rewards = 0
            moves = 0
            steps_taken = 0
            # Create initial matrix table
            initial_matrix = self.convert_to_matrix(obs)
            matrix_table, score_label, steps_label = self.visualize_matrix(initial_matrix, rewards, steps_taken)
            self.play(Create(matrix_table), Write(score_label), Write(steps_label))
            #while not done and moves < max_steps:
            while not done:
                next_state = env.board
                action = model.predict(obs, state=next_state)
                obs, reward, done, info = env.step(action[0])
                rewards += reward
                reward2 = rewards - reward
                moves += 1
                steps_taken=moves


                #Create the updated matrix table
                game_matrix = self.convert_to_matrix(next_state)
                updated_matrix_table, updated_score_label, updated_steps_label = self.visualize_matrix(game_matrix, reward2, steps_taken)


                #Transition to the updated matrix with colors
                self.play(ReplacementTransform(matrix_table, updated_matrix_table),
                        ReplacementTransform(score_label, updated_score_label),
                        ReplacementTransform(steps_label, updated_steps_label))


                matrix_table = updated_matrix_table
                score_label = updated_score_label
                steps_label = updated_steps_label
                if(moves%100==0):
                    print("_______________________________________________________________\n")
                    print('This is game: {}'.format(iteration))
                    print('The current move counter is : {}'.format(moves))
                    print('The current reward value is : {}'.format(rewards))
                rewards2_list.append(rewards)
                rewards_sum+=rewards
                moves_sum+= moves
            rewards_average=rewards_sum/1
            moves_average= moves_sum/1
            rewards2_list.sort()
            max_score=rewards2_list[1]
            print("Average Games Moves: {}".format(moves_average))
            print("Average Games Score: {}".format(rewards_average))
            print('Maximum Score :{}'.format(max_score))
            self.wait(1)  # Wait for 1 seconds at the end


    def visualize_matrix(self, matrix, score, steps_taken):
        elements = []
        for i, row in enumerate(matrix):
            elements.append(row)


        matrix_table = IntegerTable(elements, include_outer_lines=True).set_color(GREY)
        score_label = Text(f"Score: {score}",font_size = 24).next_to(matrix_table, RIGHT, buff=0.5).set_color(ORANGE)
        steps_label = Text(f"Steps: {steps_taken}",font_size = 30).next_to(score_label, DOWN, buff=0.5).set_color(ORANGE)


        return matrix_table, score_label, steps_label


    def convert_to_matrix(self, state):
        # Convert the flattened state to a 4x4 matrix
        return np.array(state).reshape((4, 4))


# Create an instance of the scene
scene = MatrixVisualization()


!manim -ql --disable_caching MatrixVisualization
!manim -pql --resolution 480,360 --disable_caching MatrixVisualization
!manim -pql --resolution 480,360 --disable_caching scene
!manim -ql --disable_caching scene


scene.render()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Manim Community v0.18.0

Traceback (most recent call last):
  File "/usr/local/bin/manim", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/manim/cli/render/commands.py", line 111, in render
    for SceneClass in scene_classes_from_file(file):
  File "/usr/local/lib/python3.10/dist-packages/manim/utils/module_ops.py", li

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


[12/05/23 17:04:40] INFO     Animation 0 : Partial movie file written in                   ]8;id=8843;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=958604;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py#527\527]8;;\
                             '/content/media/videos/1080p60/partial_movie_files/MatrixVisu                         
                             alization/3977891868_1914099888_223132457.mp4'                                        

INFO:manim:Animation 0 : Partial movie file written in '/content/media/videos/1080p60/partial_movie_files/MatrixVisualization/3977891868_1914099888_223132457.mp4'


...

INFO:manim:
File ready at '/content/media/videos/1080p60/MatrixVisualization.mp4'



                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=31464;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=418151;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene_file_writer.py#707\707]8;;\
                             manim has removed the 807 oldest file(s). You can change this                         
                             behaviour by changing max_files_cached in config.                                     

INFO:manim:The partial movie directory is full (> 100 files). Therefore, manim has removed the 807 oldest file(s). You can change this behaviour by changing max_files_cached in config.


[12/05/23 18:37:02] INFO     Rendered MatrixVisualization                                              ]8;id=461307;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=543483;file:///usr/local/lib/python3.10/dist-packages/manim/scene/scene.py#241\241]8;;\
                             Played 807 animations                                                                 

INFO:manim:Rendered MatrixVisualization
Played 807 animations
